In [1]:
import hax
hax.__version__

hax.init(experiment='XENON1T',
         pax_version_policy='loose',
         sc_api_key='ssn1sslkvdhittoywjk88w9cmbxf4fmprn68r8hm',
         force_reload=False,
         raw_data_access_mode = 'local',
         raw_data_local_path = ['/project/lgrandi/xenon1t/raw_for_waveforms/'], 
         main_data_paths= ['/dali/lgrandi/xenon1t/processed/pax_v6.10.1',''],
         minitree_paths= ['/project2/lgrandi/zhut/minitrees/pax_v6.10.1',  # New data from dali and 6.0.2 version of MultipleS2Peaks
                          '/project2/lgrandi/zhut/minitrees/pax_v6.10.0',
                          '/project2/lgrandi/xenon1t/minitrees/pax_v6.10.0',
                          '/project2/lgrandi/xenon1t/minitrees/latest'],
         use_rundb=True,
         make_minitree=True,
         minitree_caching=True,)

In [2]:
%run ./Initialize_tianyu.ipynb

In [4]:
%load_ext autoreload

In [8]:
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())

In [23]:
%aimport TwoLevel
%aimport MultipleS2Peaks

In [24]:
%autoreload 1

In [25]:
MultipleS2Corrections = MultipleS2Peaks.MultipleS2Corrections
MultipleS2Peaks = MultipleS2Peaks.MultipleS2Peaks

In [26]:
#%%prun -D MultipleS2PeaksTreeMaker.prof
testp = hax.minitrees.load('171005_0423', [MultipleS2Peaks])
teste = hax.minitrees.load('171005_0423', [MultipleS2Corrections])

In [28]:
testp.columns, teste.columns

(Index(['area', 'area_fraction_top', 'carea_nn', 'carea_tpf', 'center_time',
        'drift_time', 'event_number', 'goodness_of_fit_nn',
        'goodness_of_fit_tpf', 'index_of_maximum', 'not_interaction',
        'not_interaction_depth', 'not_interaction_pattern',
        'not_interaction_width', 'range_50p_area', 'run_number', 's1',
        's1_xyz_correction_nn_fdc_3d', 's1_xyz_correction_tpf_fdc_2d', 's2',
        's2_lifetime_correction', 's2_xy_nn_correction_tot',
        's2_xy_tpf_correction_tot', 'weighted_s1_xyz_correction_nn_fdc_3d',
        'weighted_s1_xyz_correction_tpf_fdc_2d', 'x_nn', 'x_tpf', 'y_nn',
        'y_tpf', 'z'],
       dtype='object'),
 Index(['cs1_multi_peak', 'cs2_multi_peak', 'cs2b_multi_peak',
        'cs2t_multi_peak', 'event_number', 'n_multi_peak', 'run_number',
        's2_multi_peak', 'x_nn_multi_peak', 'x_tpf_multi_peak',
        'y_nn_multi_peak', 'y_tpf_multi_peak'],
       dtype='object'))